# 파트 8: 중간 지식 베이스

파트 7에서는 속도와 비용에 최적화된 최소 추론 노력을 경험했습니다. 이 파트에서는 현재 사용 가능한 가장 높은 수준의 노력인 **중간 추론 노력**을 소개합니다. 이 노력은 원래 결과가 최적이 아니라고 판단될 때 검색 단계를 추가합니다.

## 단계 1: 환경 변수 로드

아래 셀을 실행하여 Azure 리소스의 구성을 로드하고, 생성된 **.venv(3.11.9)** 환경을 선택하세요.

이번에는 균형 잡힌 추론 기능을 가진 지식 베이스를 생성합니다.

> **⚠️ 문제 해결**
>
> 코드 셀이 멈추고 계속 돌아가면 노트북 상단 툴바에서 **Restart**를 선택하세요. 몇 번 시도해도 문제가 지속되면 VS Code를 완전히 닫고 다시 여세요.

In [ ]:
import os

from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

load_dotenv(override=True) # take environment variables from .env.

# Azure AI Search configuration
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"])

# Knowledge base name
knowledge_base_name = "medium-knowledge-base"

# Azure OpenAI configuration
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
azure_openai_chatgpt_deployment = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT", "gpt-4.1")
azure_openai_chatgpt_model_name = os.getenv("AZURE_OPENAI_CHATGPT_MODEL_NAME", "gpt-4.1")

print("Environment variables loaded")

## 단계 2: 중간 추론 지식 베이스 생성

아래 코드는 `KnowledgeRetrievalMediumReasoningEffort`를 사용하여 지식 베이스를 생성합니다. 최소 추론과 달리, 이 구성에는 적당한 쿼리 분해와 답변 합성을 위한 Azure OpenAI가 포함됩니다.

중간 추론은 쿼리 계획 및 답변 합성을 활성화하기 위해 Azure OpenAI 모델을 다시 가져오고, 원시 청크 대신 합성된 답변을 반환하기 위해 `ANSWER_SYNTHESIS` 출력 모드를 사용합니다. 구성에는 자연어로 소스 선택을 안내하는 검색 지침도 포함됩니다.

이 균형 잡힌 접근 방식은 합리적인 성능과 비용 효율성을 유지하면서 최소 노력보다 더 나은 추론 기능을 제공합니다.

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import AzureOpenAIVectorizerParameters, KnowledgeBase, KnowledgeBaseAzureOpenAIModel, KnowledgeRetrievalMediumReasoningEffort, KnowledgeRetrievalOutputMode, KnowledgeSourceReference, WebKnowledgeSource

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)

aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=azure_openai_endpoint,
    api_key=azure_openai_key,
    deployment_name=azure_openai_chatgpt_deployment,
    model_name=azure_openai_chatgpt_model_name,
)

ks = WebKnowledgeSource(
    name="web-knowledge-source",
    description="Knowledge source for Web"
)
index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"Knowledge source '{ks.name}' created or updated successfully.")

knowledge_base = KnowledgeBase(
    name=knowledge_base_name,
    models=[KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(name="hrdocs-knowledge-source"),
        KnowledgeSourceReference(name="web-knowledge-source"),
    ],
    output_mode=KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS,
    retrieval_reasoning_effort=KnowledgeRetrievalMediumReasoningEffort(),
    retrieval_instructions="Always search hrdocs first, then web if needed."
)

index_client.create_or_update_knowledge_base(knowledge_base)
print(f"Knowledge base '{knowledge_base_name}' created or updated successfully.")

## 단계 3: 복잡한 다단계 질문 쿼리

아래 쿼리는 여러 단계가 필요한 복잡한 질문으로 중간 추론 기능을 시연합니다: Zava가 휴대용 제품을 출시한 시기 찾기, 그 해의 중국 띠 결정하기, 제품 설명하기.

이 유형의 다단계 추론은 중간 노력이 정교한 쿼리 분해와 성능 최적화의 균형을 어떻게 맞추는지 보여줍니다.

In [ ]:
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import KnowledgeBaseMessage, KnowledgeBaseMessageTextContent, KnowledgeBaseRetrievalRequest, SearchIndexKnowledgeSourceParams, WebKnowledgeSourceParams
from IPython.display import display, Markdown

knowledge_base_client = KnowledgeBaseRetrievalClient(endpoint=endpoint, knowledge_base_name=knowledge_base_name, credential=credential)
web_ks_params = WebKnowledgeSourceParams(
    knowledge_source_name="web-knowledge-source",
    include_references=True,
    include_reference_source_data=True
)
hrdocs_ks_params = SearchIndexKnowledgeSourceParams(
    knowledge_source_name="hrdocs-knowledge-source",
    include_references=True,
    include_reference_source_data=True
)
req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text="""
            Zava was founded in 1985 by visionary entrepreneurs
            In one particular year, they launched a groundbreaking product that revolutionized the industry that you could hold in your hands
            Name the Chinese Zodiac sign for that year and describe the product that was launched.                         
        """)])
    ],
    knowledge_source_params=[
        web_ks_params, hrdocs_ks_params
    ],
    include_activity=True
)

result = knowledge_base_client.retrieve(retrieval_request=req)
display(Markdown(result.response[0].content[0].text))

## 단계 4: 참조 및 인용 검토

답변을 구성하는 데 사용된 참조를 확인해봅시다. 중간 추론 노력은 검색의 철저함과 효율성의 균형을 맞추면서 상세한 인용을 제공합니다.

In [ ]:
import json

references = json.dumps([ref.as_dict() for ref in result.references], indent=2)
print(references)

## 단계 5: 추론 활동 분석

아래 셀을 실행하여 중간 추론 노력이 복잡한 쿼리를 어떻게 처리했는지 보여주는 활동 로그를 확인합니다. 쿼리 분해, 검색 지침에 따른 소스 선택, 최종 답변을 합성하기 위해 수행된 추론 단계의 증거를 볼 수 있습니다.

In [ ]:
import pandas as pd

activity_types = [{"type": a.type} for a in result.activity]

df = pd.DataFrame(activity_types)

print("Activity Log Steps")
df

In [ ]:
activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2)
print("Activity Details")
print(activity_content)

## 요약

이제 정교한 추론 기능과 성능 최적화의 균형을 맞추는 중간 추론 지식 베이스를 경험했습니다.

**기억해야 할 핵심 개념:**
- `KnowledgeRetrievalMediumReasoningEffort`는 추론 정교함과 성능의 균형을 맞춥니다
- Azure OpenAI 모델은 적당한 쿼리 분해와 답변 합성을 가능하게 합니다
- `ANSWER_SYNTHESIS` 출력 모드는 적절한 인용과 함께 합성된 답변을 제공합니다
- `retrieval_instructions`는 소스 선택을 안내합니다 (먼저 hrdocs, 필요시 웹)
- 복잡한 다단계 쿼리는 정교한 정보 요구를 처리하는 중간 추론의 능력을 보여줍니다

### 다음 단계

🎉 **축하합니다!** LAB511 지식 베이스 워크샵의 8개 파트를 모두 완료했습니다. 이제 기본 단일 소스 쿼리부터 최적화된 추론 전략까지 Azure AI Search 지식 베이스의 전체 스펙트럼을 이해했습니다.

- [저장소](https://github.com/microsoft/ignite25-LAB511-build-agentic-knowledge-bases-next-level-rag-with-azure-ai-search)를 클론하고 스타를 눌러 더 탐색해보세요.
- 더 많은 정보는 [Azure AI Search 문서](https://learn.microsoft.com/en-us/azure/search/)를 확인하세요.
- 이 랩에 대한 피드백을 공유해주세요!

이제 Azure AI Search를 사용하여 특정 요구에 맞춘 고급 에이전틱 지식 베이스를 구축할 준비가 되었습니다!